## Compile test contract

In [4]:
import subprocess
solc = subprocess.Popen(['npx', 'solcjs', '--version'], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
print(solc.communicate())

contract_source = b"""
// SPDX-License-Identifier: GPL-3.0
pragma solidity >=0.4.16 <0.9.0;

contract SimpleStorage {
    uint storedData;

    function set(uint x) public {
        storedData = x;
    }

    function get() public view returns (uint) {
        return storedData;
    }
}
"""

# create tmp file to compile
with open('tmp.sol', 'wb') as f:
    f.write(contract_source)

# compile contract
solc = subprocess.run('npx solcjs --optimize --bin --abi tmp.sol', shell=True, check=True)

# extract artifacts
with open('tmp_sol_SimpleStorage.bin', 'r') as bin:
    contract_bin = bin.read()
with open('tmp_sol_SimpleStorage.abi', 'r') as abi:
    contract_abi = abi.read()

# remove tmp files
import glob, os
for f in glob.glob("tmp*"):
    os.remove(f)

print('BIN:', contract_bin)
print('ABI:', contract_abi)

(b'0.8.0+commit.c7dfd78e.Emscripten.clang\n', None)
BIN: 608060405234801561001057600080fd5b5060c38061001f6000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c806360fe47b11460375780636d4ce63c146048575b600080fd5b60466042366004606d565b6062565b005b604e6067565b604051605991906084565b60405180910390f35b600055565b60005490565b600060208284031215607d578081fd5b5035919050565b9081526020019056fea26469706673582212208f3a7502510a9a92e501e850398317afae3357045dfb3e45216741a917f43ad064736f6c63430008000033
ABI: [{"inputs":[],"name":"get","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"x","type":"uint256"}],"name":"set","outputs":[],"stateMutability":"nonpayable","type":"function"}]


## 1. Pure pyevm way (WIP)

### Imports

In [27]:
from eth_keys import keys
from eth_utils import decode_hex
from eth_typing import Address
from eth import constants
from eth.chains.base import MiningChain
from eth.consensus.pow import mine_pow_nonce
from eth.vm.forks.byzantium import ByzantiumVM
from eth.db.atomic import AtomicDB

### Chain genesis settings

In [28]:
# VM params
GENESIS_PARAMS = {
    'parent_hash': constants.GENESIS_PARENT_HASH,
    'uncles_hash': constants.EMPTY_UNCLE_HASH,
    'coinbase': constants.ZERO_ADDRESS,
    'transaction_root': constants.BLANK_ROOT_HASH,
    'receipt_root': constants.BLANK_ROOT_HASH,
    'difficulty': 1,
    'block_number': constants.GENESIS_BLOCK_NUMBER,
    'gas_limit': 3141592,
    'timestamp': 1514764800,
    'extra_data': constants.GENESIS_EXTRA_DATA,
    'nonce': constants.GENESIS_NONCE
}

# VM genesis state
DEPLOYER_PRIVATE_KEY = keys.PrivateKey(
    decode_hex('0x45a915e4d060149eb4365960e6a7a45f334393093061116b197e3240065ff2d8')
)
DEPLOYER_ADDRESS = Address(SENDER_PRIVATE_KEY.public_key.to_canonical_address())
DEFAULT_INITIAL_BALANCE = to_wei(10000, 'ether')

GENESIS_STATE = {
    DEPLOYER_ADDRESS: {
        "balance": DEFAULT_INITIAL_BALANCE,
        "nonce": 0,
        "code": b'',
        "storage": {}
    }
}

### Chain init

In [46]:
klass = MiningChain.configure(
    __name__='TestChain',
    vm_configuration=(
        (constants.GENESIS_BLOCK_NUMBER, ByzantiumVM),
    ))
chain = klass.from_genesis(AtomicDB(), GENESIS_PARAMS, GENESIS_STATE)

### Check deployer's balance

In [47]:
mock_address_balance = chain.get_vm().state.get_balance(DEPLOYER_ADDRESS)

print("The balance of address {} is {} wei".format(
    encode_hex(DEPLOYER_ADDRESS),
    mock_address_balance)
)

The balance of address 0xa94f5374fce5edbc8e2a8697c15331677e6ebf0b is 10000000000000000000000 wei


### Deploy contract

In [89]:
vm = chain.get_vm()
nonce = vm.state.get_nonce(DEPLOYER_ADDRESS)

tx = vm.create_unsigned_transaction(
    nonce=nonce,
    gas_price=0,
    gas=100000,
    to=constants.ZERO_ADDRESS,
    value=0,
    data=contract_bin,
)
signed_tx = tx.as_signed_transaction(DEPLOYER_PRIVATE_KEY)

new_block, receipt, computation = chain.apply_transaction(signed_tx)

print(new_block)
print(receipt)
print(computation)

int.from_bytes(computation.output, byteorder='big')

Block #4-0x6b8d..9e23
Receipt(state_root=b'\x01', gas_used=51736, bloom=0, logs=())


0

### Mine blocks

In [86]:
block_result = chain.get_vm().finalize_block(chain.get_block())
block = block_result.block

# based on mining_hash, block number and difficulty we can perform
# the actual Proof of Work (PoW) mechanism to mine the correct
# nonce and mix_hash for this block
nonce, mix_hash = mine_pow_nonce(
    block.number,
    block.header.mining_hash,
    block.header.difficulty
)

chain.mine_block(mix_hash=mix_hash, nonce=nonce)

<ByzantiumBlock(#Block #3-0xd821..3cdf)>

## 2. PyEVM + web3

### Imports

In [1]:
from web3 import Web3
from web3.providers.eth_tester import EthereumTesterProvider
from eth_tester import EthereumTester
from eth_tester import PyEVMBackend

### Just connect web3 to PyEVMBackend to get web3 instance

In [2]:
w3 = Web3(EthereumTesterProvider(EthereumTester(PyEVMBackend())))

### Deploy contract

In [5]:
contract = w3.eth.contract(bytecode=contract_bin, abi=contract_abi)
tx_hash = contract.constructor().transact()
tx_rcpt = w3.eth.waitForTransactionReceipt(tx_hash)
contract_address = tx_rcpt.contractAddress

test_contract = w3.eth.contract(
    address=contract_address,
    abi=contract_abi,
)

### Do anything

In [6]:
tx_hash = test_contract.functions.set(1).transact()
tx_rcpt = w3.eth.waitForTransactionReceipt(tx_hash)
print(test_contract.functions.get().call())

tx_hash = test_contract.functions.set(2).transact()
tx_rcpt = w3.eth.waitForTransactionReceipt(tx_hash)
print(test_contract.functions.get().call())

tx_hash = test_contract.functions.set(3).transact()
tx_rcpt = w3.eth.waitForTransactionReceipt(tx_hash)
print(test_contract.functions.get().call())

gas = test_contract.functions.set(2).estimateGas()
print('Gas cost execution:', gas)

1
2
3
Gas cost execution: 33381
